Read excel file sheet to dataframe  
Keep columns and rows with at least 70% Non-NaN values  
Read column 'symbol' to list 'symbols'  
For each symbol:  
&emsp;Read OHLCV csv file  
&emsp;Get date_close symbol's close  
&emsp;Calculate OBV_slope on date_end  
&emsp;Append symbol, date_close, close, date_end, OBV_slope to dict data  
Create df_out with data  
Output df_out to csv file on desktop  


In [23]:
# f_excel = r'C:\Users\ping\Desktop\vanguard_2020-01-15.xlsx'  # excel file path
f_excel = r'C:\Users\ping\Desktop\get_symbols_close.xlsx'  # excel file path
sh_excel = 'Sheet1'  # sheet
col_to_list = 'symbol'  # convert symbol column to list
date_close = '2021-02-18'  # date to get close price

In [24]:
import pandas as pd

pd.set_option("display.max_columns", 12)
pd.set_option("display.max_colwidth", 15)
pd.set_option("display.precision", 3)
pd.set_option("display.max_rows", 30)
pd.set_option("display.width", 180)

df = pd.read_excel(f_excel, sheet_name=sh_excel, engine='openpyxl')
print(f'df before drop NaN:\n{df}\n')
# keep columns with 70% Non-NaN values
df = df.dropna(thresh=df.shape[0]*0.7, axis=1)
# keep rows with 70% Non-NaN values
df = df.dropna(thresh=df.shape[1]*0.7, axis=0)
print(f'df after drop NaN:\n{df}\n')

df before drop NaN:
   symbol
0    AAPL
1   ADYEY
2    AMZN
3     API
4    BABA
..    ...
50    TSM
51   TWLO
52   TWST
53   VRTX
54      Z

[55 rows x 1 columns]

df after drop NaN:
   symbol
0    AAPL
1   ADYEY
2    AMZN
3     API
4    BABA
..    ...
50    TSM
51   TWLO
52   TWST
53   VRTX
54      Z

[55 rows x 1 columns]



In [25]:
symbols = df[col_to_list].to_list()  # convert symbols in column to list
print(f'symbols ({len(symbols)}): {symbols}')

symbols (55): ['AAPL', 'ADYEY', 'AMZN', 'API', 'BABA', 'BIDU', 'BYDDY', 'CAT', 'CDNA', 'CRSP', 'DE', 'EXAS', 'FATE', 'FB', 'FSLY', 'GBTC', 'GOOG', 'ICE', 'IOVA', 'IRDM', 'JD', 'KTOS', 'MELI', 'MTLS', 'NFLX', 'NNDM', 'NVS', 'NVTA', 'NXPI', 'PACB', 'PINS', 'PRLB', 'PSNL', 'PSTG', 'PYPL', 'REGN', 'RHHBY', 'ROKU', 'SE', 'SHOP', 'SI', 'SNAP', 'SPCE', 'SPOT', 'SQ', 'TAK', 'TCEHY', 'TDOC', 'TRMB', 'TSLA', 'TSM', 'TWLO', 'TWST', 'VRTX', 'Z']


In [26]:
from util import OBV_calc, pickle_load, dates_within_limits
path_OHLCV = "C:/Users/ping/Google Drive/stocks/MktCap2b_AUMtop1200/OHLCV/"
col_names = ["date", "open", "high", "low", "close", "volume"]

# get date index of 'XOM'
index_symbol = 'XOM'
df = pd.read_csv(
    path_OHLCV + index_symbol + '.csv',
    names=col_names,
    parse_dates=True,
    index_col=0,)
date_index_all_dates = df.index

In [27]:
import datetime as dt

# last date of df 
# date_end_limit = '2021-02-16'
date_end_limit = dt.date.today().strftime("%Y-%m-%d")

iloc_offset = 252  # number of days to plot
date_start_limit = None
# start and end dates of df
date_start, date_end, iloc_date_start, iloc_date_end = \
    dates_within_limits(date_index_all_dates, date_start_limit,
                        date_end_limit, iloc_offset)

In [28]:
path_OHLCV = "C:/Users/ping/Google Drive/stocks/MktCap2b_AUMtop1200/OHLCV/"
col_names = ["date", "open", "high", "low", "close", "volume"]

# list of dicts to build dataframe
data = []
for symbol in symbols:
    df = pd.read_csv(
        path_OHLCV + symbol + '.csv',
        names=col_names,
        parse_dates=True,
        index_col=0,)
    df_tmp = df[date_start:date_end]
    # average OBV_EMA difference for the last tail_pd days, default is 5 days
    tail_pd5 = 5
    tail_pd30 = 30
    OBV, OBV_EMA, OBV_slope_5 = OBV_calc(df, symbol, EMA_pd=10, tail_pd=tail_pd5, norm_pd=30)
    OBV, OBV_EMA, OBV_slope_30 = OBV_calc(df, symbol, EMA_pd=10, tail_pd=tail_pd30, norm_pd=30)
    try:
        idx_row = df.index.get_loc(date_close)
        close = df.iloc[idx_row].close
        dict_tmp = {'symbol': symbol, 'date_close': date_close,  'close': close, 'date_OBV_slope': date_end, \
            f'OBV_slope_avg_last_{tail_pd30}': OBV_slope_30, f'OBV_slope_avg_last_{tail_pd5}': OBV_slope_5}
        data.append(dict_tmp)
    except KeyError:
        print(f'ERROR: No "close price" for {symbol:<{6}} on {my_date}')

In [29]:
df_out = pd.DataFrame(data)
print(df_out)

   symbol  date_close    close date_OBV_slope  OBV_slope_avg_last_30  OBV_slope_avg_last_5
0    AAPL  2021-02-18   129.71     2021-02-18          -0.048                 -0.294      
1   ADYEY  2021-02-18    52.89     2021-02-18             NaN                  0.642      
2    AMZN  2021-02-18  3328.23     2021-02-18           0.121                 -0.065      
3     API  2021-02-18    92.54     2021-02-18           0.239                 -0.062      
4    BABA  2021-02-18   264.51     2021-02-18           0.187                  0.256      
..    ...         ...      ...            ...             ...                    ...      
50    TSM  2021-02-18   134.63     2021-02-18           0.172                  0.193      
51   TWLO  2021-02-18   443.49     2021-02-18           0.070                  0.137      
52   TWST  2021-02-18   149.04     2021-02-18          -0.028                 -0.328      
53   VRTX  2021-02-18   209.83     2021-02-18          -0.281                 -0.345      

In [30]:
df_out['OBV_slope_weighted'] = 0.5 * df_out.OBV_slope_avg_last_30 + 0.5 * df_out.OBV_slope_avg_last_5
print(df_out) 

   symbol  date_close    close date_OBV_slope  OBV_slope_avg_last_30  OBV_slope_avg_last_5  OBV_slope_weighted
0    AAPL  2021-02-18   129.71     2021-02-18          -0.048                 -0.294                -0.171    
1   ADYEY  2021-02-18    52.89     2021-02-18             NaN                  0.642                   NaN    
2    AMZN  2021-02-18  3328.23     2021-02-18           0.121                 -0.065                 0.028    
3     API  2021-02-18    92.54     2021-02-18           0.239                 -0.062                 0.088    
4    BABA  2021-02-18   264.51     2021-02-18           0.187                  0.256                 0.222    
..    ...         ...      ...            ...             ...                    ...                   ...    
50    TSM  2021-02-18   134.63     2021-02-18           0.172                  0.193                 0.183    
51   TWLO  2021-02-18   443.49     2021-02-18           0.070                  0.137                 0.103    
5

In [32]:
# largest OBV_slope_weighted = highest rank
df_out['rank_OBV_slope'] = df_out.OBV_slope_weighted.rank()
df_out['rank_pct'] = df_out.OBV_slope_weighted.rank(pct=True)
print(df_out.sort_values(by='rank_OBV_slope', ascending=False)) 

   symbol  date_close   close date_OBV_slope  OBV_slope_avg_last_30  OBV_slope_avg_last_5  OBV_slope_weighted  rank_OBV_slope  rank_pct
40     SI  2021-02-18  153.75     2021-02-18           0.576                  0.478                 0.527                54.0     1.000
54      Z  2021-02-18  181.25     2021-02-18           0.213                  0.796                 0.505                53.0     0.981
5    BIDU  2021-02-18  298.01     2021-02-18           0.372                  0.565                 0.468                52.0     0.963
34   PYPL  2021-02-18  290.81     2021-02-18           0.205                  0.630                 0.417                51.0     0.944
33   PSTG  2021-02-18   27.11     2021-02-18           0.379                  0.453                 0.416                50.0     0.926
..    ...         ...     ...            ...             ...                    ...                   ...                 ...       ...
17    ICE  2021-02-18  113.86     2021-02-18    

In [34]:
pd.set_option("display.max_rows", 100)
print(df_out.sort_values(by='rank_OBV_slope', ascending=False).head(60)) 

   symbol  date_close     close date_OBV_slope  OBV_slope_avg_last_30  OBV_slope_avg_last_5  OBV_slope_weighted  rank_OBV_slope  rank_pct
40     SI  2021-02-18   153.750     2021-02-18           0.576                  0.478                 0.527                54.0     1.000
54      Z  2021-02-18   181.250     2021-02-18           0.213                  0.796                 0.505                53.0     0.981
5    BIDU  2021-02-18   298.010     2021-02-18           0.372                  0.565                 0.468                52.0     0.963
34   PYPL  2021-02-18   290.810     2021-02-18           0.205                  0.630                 0.417                51.0     0.944
33   PSTG  2021-02-18    27.110     2021-02-18           0.379                  0.453                 0.416                50.0     0.926
30   PINS  2021-02-18    85.990     2021-02-18           0.197                  0.503                 0.350                49.0     0.907
38     SE  2021-02-18   267.500   

In [35]:
csv_out = r'C:\Users\ping\Desktop\df_out.csv'
df_out.to_csv(csv_out)
print(df_out)


   symbol  date_close     close date_OBV_slope  OBV_slope_avg_last_30  OBV_slope_avg_last_5  OBV_slope_weighted  rank_OBV_slope  rank_pct
0    AAPL  2021-02-18   129.710     2021-02-18          -0.048                 -0.294                -0.171                 6.0     0.111
1   ADYEY  2021-02-18    52.890     2021-02-18             NaN                  0.642                   NaN                 NaN       NaN
2    AMZN  2021-02-18  3328.230     2021-02-18           0.121                 -0.065                 0.028                22.0     0.407
3     API  2021-02-18    92.540     2021-02-18           0.239                 -0.062                 0.088                26.0     0.481
4    BABA  2021-02-18   264.510     2021-02-18           0.187                  0.256                 0.222                39.0     0.722
5    BIDU  2021-02-18   298.010     2021-02-18           0.372                  0.565                 0.468                52.0     0.963
6   BYDDY  2021-02-18    64.790   